In [2]:
!nvidia-smi

Tue Apr 22 17:21:58 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P0             27W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
!pip install ultralytics

In [4]:
!pip install -U "ray[default]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 51.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 82.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 28.7 MB/s eta 0:00:00


In [5]:
# Install the Roboflow library
!pip install roboflow

In [6]:
!git clone https://github.com/THU-MIG/yolov10.git
%cd yolov10

fatal: destination path 'yolov10' already exists and is not an empty directory.
/kaggle/working/yolov10


In [7]:
import os
import urllib.request
#Create a directory for the weights in the current working directory
weights_dir = os.path.join(os.getcwd(), 'weights')
os.makedirs(weights_dir, exist_ok = True)
#urls of the weights file
urls = ["https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10n.pt",
        "https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10s.pt",
        "https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10m.pt",
        "https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10b.pt",
        "https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10l.pt",
        "https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10x.pt"]

#Download each file
for url in urls:
  filename = os.path.basename(url)
  filepath = os.path.join(weights_dir, filename)
  urllib.request.urlretrieve(url, filepath)
  print(f"Downloaded: {filepath}")


Downloaded: /kaggle/working/yolov10/weights/yolov10n.pt
Downloaded: /kaggle/working/yolov10/weights/yolov10s.pt
Downloaded: /kaggle/working/yolov10/weights/yolov10m.pt
Downloaded: /kaggle/working/yolov10/weights/yolov10b.pt
Downloaded: /kaggle/working/yolov10/weights/yolov10l.pt
Downloaded: /kaggle/working/yolov10/weights/yolov10x.pt


In [9]:
from roboflow import Roboflow
rf = Roboflow(api_key="eperym0ysR30jS6uFipI")
project = rf.workspace("phfs").project("phfs-factory-person-fire-and-smoke-merge")
version = project.version(2)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...


In [10]:
yaml_path = "/kaggle/working/yolov10/phfs-factory-person-fire-and-smoke-merge-2/data.yaml"  # Adjust the path if needed

# Read the existing YAML file
import yaml

with open(yaml_path, "r") as file:
    data = yaml.safe_load(file)

# Update the paths
data["train"] = "/kaggle/working/yolov10/phfs-factory-person-fire-and-smoke-merge-2/train"
data["val"] = "/kaggle/working/yolov10/phfs-factory-person-fire-and-smoke-merge-2/valid"
data["test"] = "/kaggle/working/yolov10/phfs-factory-person-fire-and-smoke-merge-2/test"

# Save the updated YAML file
with open(yaml_path, "w") as file:
    yaml.dump(data, file, default_flow_style=False)

print("✅ data.yaml updated successfully!")

✅ data.yaml updated successfully!


In [12]:
!yolo task=detect mode=train epochs=150 batch=32 plots=True model='weights/yolov10s.pt' data='/kaggle/working/yolov10/phfs-factory-person-fire-and-smoke-merge-2/data.yaml'

Ultralytics 8.3.113 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=weights/yolov10s.pt, data=/kaggle/working/yolov10/phfs-factory-person-fire-and-smoke-merge-2/data.yaml, epochs=150, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_c